# Extract one-language records from Wikipedia-based Image Text (WIT) Dataset
For WIT dataset, see https://github.com/google-research-datasets/wit and https://ai.googleblog.com/2021/09/announcing-wit-wikipedia-based-image.html .

In [ ]:
import os
import re
from tqdm import tqdm

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Directory to save output files
save_dir = '/content/drive/MyDrive/wit_data'
os.makedirs(save_dir, exist_ok=True)

## Download files and Eextract target language

In [ ]:
# All the languages included in WIT dataset
language_set = {
    'af', 'an', 'ar', 'arz', 'ast', 'az', 'azb', 'ba', 'bar', 'be', 'be-tarask',
    'bg', 'bn', 'br', 'bs', 'ca', 'ce', 'ceb', 'ckb', 'cs', 'cv', 'cy', 'da',
    'de', 'el', 'en', 'eo', 'es', 'et', 'eu', 'fa', 'fi', 'fil', 'fr', 'fy',
    'ga', 'gl', 'hi', 'hr', 'hsb', 'ht', 'hu', 'hy', 'ia', 'id', 'io', 'is',
    'it', 'iw', 'ja', 'jv', 'ka', 'kk', 'kn', 'ko', 'la', 'lah', 'lb', 'lmo',
    'lt', 'lv', 'mg', 'mk', 'ml', 'mn', 'mr', 'ms', 'my', 'nan', 'nds', 'ne',
    'nl', 'nn', 'no', 'nv', 'oc', 'pa', 'pl', 'pt', 'qu', 'ro', 'ru', 'sco',
    'si', 'sk', 'sl', 'sq', 'sr', 'sr-Latn', 'sv', 'sw', 'ta', 'te', 'tg', 'th',
    'tr', 'tt', 'uk', 'ur', 'uz', 'vec', 'vi', 'vo', 'war', 'xmf', 'yue',
    'zh', 'zh-TW'
    }

all_pattern = re.compile('|'.join([lang + '\t' for lang in language_set]))

In [ ]:
# Target language to extract
lang = 'ja'
target_pattern = re.compile(lang + '\t')

length_list = []
for idx in range(10):
  data_index = str(idx).zfill(5)
  gz_file_name = "wit_v1.train.all-" + data_index + "-of-00010.tsv.gz"

  # download and unzip
  download_link = "https://storage.googleapis.com/gresearch/wit/" + gz_file_name
  !wget $download_link
  !gzip -d $gz_file_name

  # extract the target language
  file_name = gz_file_name.replace('.gz', '')
  data = []
  with open(file_name, 'r') as f:
    col_names = next(f)
    tmp_line = ""
    for raw_line in tqdm(f):
      if all_pattern.match(raw_line):
        # A new record starts in this line.
        # Check if the **previous** line is the target language
        if target_pattern.match(tmp_line):
          data.append(tmp_line)
        tmp_line = raw_line
      else:
        # This line is continued from the previous line.
        tmp_line = tmp_line + '\n' + raw_line
    # Check if the last line is the target language
    if target_pattern.match(tmp_line):
      data.append(tmp_line)

  # save the extracted file
  save_path = os.path.join(save_dir, file_name.replace('.tsv', '_' + lang + '.tsv'))
  with open(save_path, 'w') as f:
    f.write(col_names)
    f.writelines(data)

  # Remove the original file
  !rm $file_name

  length_list.append(len(data))
  print("#############################################")
  print("File ", file_name)
  print("Length ", len(data))
  print("#############################################")

print("Total # of records: ", sum(length_list))

## Check the content

In [ ]:
import pandas as pd

# Load one tsv file.
data_index = '00000'
file_name = "wit_v1.train.all-" + data_index + "-of-00010_" + lang + ".tsv"
file_path = os.path.join(save_dir, file_name)
df = pd.read_table(file_path)
df.shape

(108048, 17)

In [ ]:
df_t = df[df['is_main_image'] == True]
print(len(df_t))
df_t.head(10)

28335


,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
1,ja,https://ja.wikipedia.org/wiki/%E3%83%9F%E3%83%...,https://upload.wikimedia.org/wikipedia/commons...,ミナス・ジェライス級戦艦,NaN,ミナス・ジェライス級戦艦,NaN,English: Minas Gerais sailing soon after her c...,NaN,image/jpeg,1008,1830,True,False,False,ミナス・ジェライス級戦艦とはブラジル海軍が南アメリカ諸国で最初に購入した弩級戦艦の艦級で竣工...,ミナス・ジェライス（Minas Gerais）級戦艦とはブラジル海軍が南アメリカ諸国で最初に...
8,ja,https://ja.wikipedia.org/wiki/%E3%82%A4%E3%82%...,http://upload.wikimedia.org/wikipedia/commons/...,イエラペトラ,NaN,イエラペトラ,NaN,Locator map of Ierapetra municipality (Δήμος Ι...,イエラペトラの位置の位置図,image/png,513,700,True,False,False,イエラペトラは、ギリシャのクレタ島東部、ラシティ県の町である。古代はヒエラプトナと呼ばれた。...,イエラペトラ（ギリシャ語:Ιεράπετρα）は、ギリシャのクレタ島東部、ラシティ県の町であ...
9,ja,https://ja.wikipedia.org/wiki/%E5%B0%8F%E8%8C%...,https://upload.wikimedia.org/wikipedia/commons...,小茂田浜神社,NaN,小茂田浜神社,NaN,日本語: 小茂田神社の鳥居,小茂田神社の鳥居,image/jpeg,1920,2560,True,True,True,小茂田浜神社は、長崎県対馬市厳原に鎮座する神社である。対馬西南海岸に位置し、元寇の際に元・高...,小茂田浜神社（こもだはまじんじゃ）は、長崎県対馬市厳原に鎮座する神社である。対馬西南海岸に位...
10,ja,https://ja.wikipedia.org/wiki/%E3%82%A8%E3%83%...,http://upload.wikimedia.org/wikipedia/commons/...,エトミデート,NaN,エトミデート,NaN,English: Etomidat_Structural_Formulae,NaN,image/png,1616,2574,True,False,False,エトミデート は、イミダゾール派生薬である。静脈麻酔薬として用いられる。\n\n2012年現...,エトミデート (etomidate) は、イミダゾール派生薬である。静脈麻酔薬として用いられ...
12,ja,https://ja.wikipedia.org/wiki/%E6%B4%AA%E9%9C%87,http://upload.wikimedia.org/wikipedia/commons/...,洪震,NaN,洪震,NaN,"한국어: 1943년 3월, 중국 충칭에서 열린 자유한인대회에서 대한민국임시정부 임시...",洪震,image/jpeg,535,750,True,False,False,洪震／洪鎮は、大韓民国の独立運動家。1926年7月から同年12月まで第5代大韓民国臨時政府大...,洪震／洪鎮（ホン・ジン、1877年旧暦8月27日 – 1946年9月9日）は、大韓民国の独立...
17,ja,https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%...,https://upload.wikimedia.org/wikipedia/commons...,プログレスM1-3,NaN,プログレスM1-3,STS-106から撮影,ISS Zvezda service module with docked Progress...,NaN,image/jpeg,1104,1632,True,False,False,プログレスM1-3は2000年8月6日に打ち上げられたプログレス補給船。国際宇宙ステーション...,プログレスM1-3（Progress M1-3）は2000年8月6日に打ち上げられたプログレ...
21,ja,https://ja.wikipedia.org/wiki/%E8%B6%B3%E7%AB%...,https://upload.wikimedia.org/wikipedia/commons...,足立区立第二中学校,NaN,足立区立第二中学校,NaN,撮影位置 - 東京都、足立区。English: The second junior high...,NaN,image/jpeg,2128,2832,True,True,True,足立区立第二中学校は、かつて東京都足立区千住曙町にあった公立中学校。略称は足立二中。\n\n...,足立区立第二中学校（あだちくりつ だいにちゅうがっこう）は、かつて東京都足立区千住曙町にあっ...
22,ja,https://ja.wikipedia.org/wiki/%E3%83%86%E3%83%...,http://upload.wikimedia.org/wikipedia/commons/...,テトリル,NaN,テトリル,NaN,Chemical structure of Tetryl,構造式 テトリル,image/png,943,1027,True,False,False,テトリル は、化学式 (O₂N)₃C₆H₂-NNO₂ で示される爆薬の一種である。\n\n別...,テトリル (tetryl) は、化学式 (O₂N)₃C₆H₂-N(CH₃)NO₂ で示される...
24,ja,https://ja.wikipedia.org/wiki/%E3%83%9E%E3%83%...,http://upload.wikimedia.org/wikipedia/commons/...,マレイン酸,NaN,マレイン酸,NaN,Skeletal formula of maleic acid,マレイン酸構造式,image/png,513,1100,True,False,False,マレイン酸は、鎖状不飽和ジカルボン酸のひとつである。構造式 HOOC–CH=CH–COOH ...,マレイン酸（マレインさん、Maleic acid）は、鎖状不飽和ジカルボン酸のひとつである。...
31,ja,https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%...,https://upload.wikimedia.org/wikipedia/commons...,プール (イングランド),NaN,プール (イングランド),NaN,Poole Quay,NaN,image/jpeg,1944,3456,True,False,True,プール は、イギリスの南岸の典礼カウンティ・ドーセットにある港湾都市で、行政上はタウンかつバ...,プール (Poole) は、イギリスの南岸の典礼カウンティ・ドーセットにある港湾都市で、行政...


In [ ]:
idx = 17
for k, v in zip(df.columns, df.loc[idx, :]):
  print(k, ': ', v)

language :  ja
page_url :  https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%AD%E3%82%B0%E3%83%AC%E3%82%B9M1-3
image_url :  https://upload.wikimedia.org/wikipedia/commons/0/07/ISS_Zvezda_module.jpg
page_title :  プログレスM1-3
section_title :  nan
hierarchical_section_title :  プログレスM1-3
caption_reference_description :  STS-106から撮影
caption_attribution_description :  ISS Zvezda service module with docked Progress M1-3 spacecraft, taken in September 2000 by STS-106 (NASA)
caption_alt_text_description :  nan
mime_type :  image/jpeg
original_height :  1104
original_width :  1632
is_main_image :  True
attribution_passes_lang_id :  False
page_changed_recently :  False
context_page_description :  プログレスM1-3は2000年8月6日に打ち上げられたプログレス補給船。国際宇宙ステーションへの補給活動に使用され、バージョンはProgress-M1 11F615A55、シリアル番号は251。NASAによる名称はプログレス1。

2000年8月6日16時26分42秒、バイコヌール宇宙基地のSite 1/5からソユーズ-Uロケットによって打ち上げられた。M1-3は8月8日20時12分56秒ズヴェズダモジュールにドッキングし、国際宇宙ステーションの補給に使用された初の無人宇宙補給機となった。この当時はまだ長期滞在員が存在しなかったので、貨物はSTS-106およびSTS-92のスペースシャトルミッションでISSに移動さ

In [ ]:
df_f = df[df['is_main_image'] == False]
print(len(df_f))
df_f.head(10)

79713


,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
0,ja,https://ja.wikipedia.org/wiki/%E6%84%9B%E5%AD%...,https://upload.wikimedia.org/wikipedia/commons...,愛子内親王,来歴,愛子内親王 / 来歴,お印に選ばれた、ゴヨウツツジ,日本語: シロヤシオ（白八汐、学名：Rhododendron quinquefolium B...,NaN,image/jpeg,2200,2560,False,True,True,愛子内親王は、日本の皇族。称号は敬宮、お印はゴヨウツツジ。身位は内親王。敬称は殿下。\n\n...,（各事象等における身位の表記は、当時に沿う。）\n\n2001年（平成13年）12月1日14...
2,ja,https://ja.wikipedia.org/wiki/%E4%B9%BE%E3%83%...,https://upload.wikimedia.org/wikipedia/commons...,乾ドック,ギャラリー,乾ドック / ギャラリー,NaN,English: The USS Greeneville (SSN 772) sits at...,NaN,image/jpeg,1500,2250,False,False,False,乾ドックとは、船体の検査や修理などのために水を抜くことができるドックのこと。船渠、乾船渠とも。,NaN
3,ja,https://ja.wikipedia.org/wiki/%E9%98%BF%E9%87%...,https://upload.wikimedia.org/wikipedia/commons...,阿里山森林鉄路の蒸気機関車,ギャラリー,阿里山森林鉄路の蒸気機関車 / ギャラリー,NaN,Alishan Forest Railway 28t Shay No.22 Jiji Sta...,NaN,image/jpeg,1240,1831,False,True,False,阿里山森林鉄路の蒸気機関車では阿里山森林鉄路の蒸気機関車について述べる。,NaN
4,ja,https://ja.wikipedia.org/wiki/%E3%83%8A%E3%83%...,https://upload.wikimedia.org/wikipedia/commons...,ナビー・ムーサー,ギャラリー,ナビー・ムーサー / ギャラリー,NaN,English: Inscription plate at main gate,NaN,image/jpeg,1944,2592,False,False,False,ナビー・ムーサージュデア沙漠の地名である。モーゼに関する、有名なパレスチナの民話で知られる。...,NaN
5,ja,https://ja.wikipedia.org/wiki/%E3%83%95%E3%82%...,http://upload.wikimedia.org/wikipedia/commons/...,フォルニ・ディ・ソプラ,分離集落,フォルニ・ディ・ソプラ / 行政 / 分離集落,集落,"Forni di Sopra, Friuli-Venezia Giulia, Italy",NaN,image/jpeg,1200,1600,False,False,False,フォルニ・ディ・ソプラは、イタリア共和国フリウリ＝ヴェネツィア・ジュリア州ウーディネ県にある...,コムーネは、3つのフラツィオーネ（分離集落）に分けられる。村役場はVicoに置かれている。イ...
6,ja,https://ja.wikipedia.org/wiki/%E6%96%B0%E4%B8%...,https://upload.wikimedia.org/wikipedia/commons...,新上挙母駅,駅構造,新上挙母駅 / 駅構造,NaN,日本語: Shin-uwagoromo station name board on the ...,NaN,image/jpeg,624,832,False,True,True,新上挙母駅は、愛知県豊田市司町にある愛知環状鉄道線の駅である。駅番号は11。,相対式2面2線ホームを持つ高架駅で、ホームは4両まで対応している。2008年3月17日から平...
7,ja,https://ja.wikipedia.org/wiki/%E5%B7%9D%E8%A5%...,https://upload.wikimedia.org/wikipedia/ja/0/00...,川西市,名所・旧跡・観光スポット・祭事・催事,川西市 / 名所・旧跡・観光スポット・祭事・催事,多田神社,多田神社本殿 アップロード者のコメント: 2004年(平成16年)3月12日に投稿者が撮影し...,NaN,image/jpeg,360,480,False,True,True,川西市は、兵庫県南東部の市。大阪府との県境に面する。阪神北県民局管轄区域。,多田神社\n\n満願寺 - 新西国三十三箇所観音霊場第13番 寺の位置する満願寺町は周囲を宝...
11,ja,https://ja.wikipedia.org/wiki/%E6%B0%B4%E7%B4%...,https://upload.wikimedia.org/wikipedia/commons...,水素原子におけるシュレーディンガー方程式の解,具体的な値,水素原子におけるシュレーディンガー方程式の解 / 具体的な値,NaN,English: Radial distribution functions in 2s o...,NaN,image/png,960,1280,False,True,True,本項、水素原子におけるシュレーディンガー方程式の解では、ハミルトニアンが\n\nと書ける二粒...,水素原子の波動関数のl=0～3における角因子は以下のようになる。ここで、はそれぞれ、動径方向...
13,ja,https://ja.wikipedia.org/wiki/G%27zOne,https://upload.wikimedia.org/wikipedia/commons...,G'zOne,一覧,G'zOne / 端末 / 日本（KDDI・沖縄セルラー電話(旧：IDO・DDIセルラーグル...,NaN,IS11CA 表側,NaN,image/jpeg,3000,4000,False,True,False,G'zOneは、かつてカシオ計算機およびカシオ日立モバイルコミュニケーションズ→NECカシオ...,*1： CDMA2000 1x EV-DO Rev.A対応、*2： カロリーカウンターOFF...
14,ja,https://ja.wikipedia.org/wiki/%E3%82%B9%E3%82%...,https://upload.wikimedia.org/wikipedia/commons...,スズキ・ソリオ,年表,スズキ・ソリオ / 3代目 MA26S/MA36S/MA46S型（2015年-） / 年表,NaN,日本語: MA36S型スズキ・ソリオバンディットHYBRID MVをリアから撮影。,NaN,image/jpeg,1882,2532,False,True,True,ソリオは、スズキが製造・販売するコンパクトトールワゴン。\n\n本項ではソリオの前身ブランド...,2015年\n\n8月26日 - ソリオ・ソリオ バンディット（以下、バンディット）共にフル...


In [ ]:
for k, v in zip(df.columns, df.loc[11, :]):
  print(k, ': ', v)

language :  ja
page_url :  https://ja.wikipedia.org/wiki/%E6%B0%B4%E7%B4%A0%E5%8E%9F%E5%AD%90%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8B%E3%82%B7%E3%83%A5%E3%83%AC%E3%83%BC%E3%83%87%E3%82%A3%E3%83%B3%E3%82%AC%E3%83%BC%E6%96%B9%E7%A8%8B%E5%BC%8F%E3%81%AE%E8%A7%A3
image_url :  https://upload.wikimedia.org/wikipedia/commons/4/40/R-2s-orbital.png
page_title :  水素原子におけるシュレーディンガー方程式の解
section_title :  具体的な値
hierarchical_section_title :  水素原子におけるシュレーディンガー方程式の解 / 具体的な値
caption_reference_description :  nan
caption_attribution_description :  English: Radial distribution functions in 2s orbital 日本語: 2s軌道の動径関数
caption_alt_text_description :  nan
mime_type :  image/png
original_height :  960
original_width :  1280
is_main_image :  False
attribution_passes_lang_id :  True
page_changed_recently :  True
context_page_description :  本項、水素原子におけるシュレーディンガー方程式の解では、ハミルトニアンが

と書ける二粒子系の時間非依存なシュレーディンガー方程式の厳密解を解く。

物理学的にはこれは、

質量m₀の正の電荷をもつ粒子と質量がm₁負の電荷を持つ粒子がクーロン力により結合している状況において

外力は働いておらず、

相対論的効果を考えない量子力学の範囲内で、

時間に依存し